# Resources
- [web-api](https://developer.spotify.com/documentation/web-api/)
- [Ben's API explainer](https://docs.google.com/document/d/18hQnCt2kiFXaa0fQ24pkCcVChDtCmhR7m7XpTqNSLsI/edit)
- [spotipy API](https://spotipy.readthedocs.io/en/master/#api-reference)
- [genius API](https://lyricsgenius.readthedocs.io/en/master/reference/api.html#api)

# Setup

In [17]:
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from lyricsgenius import Genius
import os
import numpy as np
import pandas as pd
from pathlib import Path
from joblib import dump, load

# Change base_dir as needed
base_dir = Path("/Users/Esh/Documents/conferences/csss2022/narrative_diffusion/")
analysis_dir = base_dir / 'analysis'
load_dotenv()


True

In [3]:
# Client credentials flow for public data
sp_client = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=os.environ["SPOTIFY_CLIENT_ID"],
        client_secret=os.environ["SPOTIFY_CLIENT_SECRET"],
    )
)

# Auth Code flow, for user data
sp_auth = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=os.environ["SPOTIFY_CLIENT_ID"],
        client_secret=os.environ["SPOTIFY_CLIENT_SECRET"],
        redirect_uri=os.environ["REDIRECT_URI"],
        scope=os.environ["SPOTIFY_SCOPE"],
    )
)

# Genius
genius = Genius(os.environ['GENIUS_ACCESS_TOKEN'])

# Example retrieving public data using spotify API

In [4]:
playlist_id = "37i9dQZF1DX59HcpGmPXYR"
playlist = sp_client.playlist(playlist_id=playlist_id)
playlist['name']


'Pride Classics'

# Example retrieving user data using spotify API

In [13]:
# Get top-artists and save it to a file
top_artists_path = analysis_dir / 'my_top_artists.joblib'
if top_artists_path.exists():
    top_artists = load(top_artists_path)
else:
    top_artists=sp_auth.current_user_top_artists(limit=50)
    dump(top_artists, top_artists_path)

In [20]:
artists_df = pd.DataFrame(top_artists['items'])
artists_df.head()

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 11200495}","[lgbtq+ hip hop, pop]",https://api.spotify.com/v1/artists/7jVv8c5Fj3E...,7jVv8c5Fj3E9VhNjxT4snq,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Lil Nas X,82,artist,spotify:artist:7jVv8c5Fj3E9VhNjxT4snq
1,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 1748542}","[chicago rap, conscious hip hop, gangster rap,...",https://api.spotify.com/v1/artists/01QTIT5P1pF...,01QTIT5P1pFP3QnnFSdsJf,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Lupe Fiasco,65,artist,spotify:artist:01QTIT5P1pFP3QnnFSdsJf
2,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 17534836}","[chicago rap, rap]",https://api.spotify.com/v1/artists/5K4W6rqBFWD...,5K4W6rqBFWDnAN6FQUkS6x,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kanye West,90,artist,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x
3,{'spotify': 'https://open.spotify.com/artist/4...,"{'href': None, 'total': 981348}","[escape room, hip hop, political hip hop]",https://api.spotify.com/v1/artists/4RnBFZRiMLR...,4RnBFZRiMLRyZy0AzzTg2C,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Run The Jewels,60,artist,spotify:artist:4RnBFZRiMLRyZy0AzzTg2C
4,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 2876685}","[conscious hip hop, east coast hip hop, gangst...",https://api.spotify.com/v1/artists/20qISvAhX20...,20qISvAhX20dpIbOOzGK3q,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Nas,72,artist,spotify:artist:20qISvAhX20dpIbOOzGK3q


# Example getting lyrics using genius API

In [5]:
artist = genius.search_artist('Kanye West', max_songs=3, sort='title')

Searching for songs by Kanye West...

Song 1: "03 Electic Relaxation"
Song 2: "12,000 Acres"
"124th" is not valid. Skipping.
Song 3: "18 Years"

Reached user-specified song limit (3).
Done. Found 3 songs.


[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]

In [11]:
# Save them to analyis folder
artist.save_lyrics()
os.rename("Lyrics_KanyeWest.json", str(analysis_dir / "Lyrics_KanyeWest.json"))


Wrote Lyrics_KanyeWest.json.
